In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np


In [152]:
from data_master_eng_ml.config import MONGODB_DATABASE,

MongoClient(host=['ac-v7jdtxc-shard-00-00.1qzsmiy.mongodb.net:27017', 'ac-v7jdtxc-shard-00-02.1qzsmiy.mongodb.net:27017', 'ac-v7jdtxc-shard-00-01.1qzsmiy.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='datamaster"', authsource='admin', replicaset='atlas-4a25lc-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x14137e990>)

In [3]:
ano = "2022"

In [110]:
from data_master_eng_ml.transformations.api_transformations import (
    fetch_game_release_dates,
    fetch_involved_companies,
    fetch_companies_info,
    fetch_multiplayer_modes,
    fetch_game_info,
    batch_fetch_age_classifications
)
from data_master_eng_ml.utils.helpers import map_age_classifications,generate_dummies


In [70]:
from data_master_eng_ml.dataset import main

In [98]:
def game_featurization(single: bool, game_id: int = 0, year: int = 2021):
    """
    Função principal que realiza a featurização do jogo.
    """
    # Obtém dados de diferentes fontes da API
    if not single:
        data_frame_games_find = fetch_game_release_dates(single=single, year=year)
        game_id = list(set(data_frame_games_find["game"].to_list()))
        games_id_list = ",".join(map(str, game_id))
    else:
        data_frame_games_find = fetch_game_release_dates(single=single, year=year,game_id=game_id)
        games_id_list = ",".join(map(str, [game_id]))
    data_frame_companies_find = fetch_involved_companies(games_id_list)
    company_id_list = list(set(data_frame_companies_find["company"].to_list()))
    data_frame_companies = fetch_companies_info(company_id_list)

    # Processa dados das empresas
    companies_join = pd.merge(
        data_frame_companies_find,
        data_frame_companies,
        left_on="company",
        right_on="id",
        how="inner",
    )
    df_companies_final = companies_join[
        ["game", "games_developed", "has_parents", "games_published", "continent_name"]
    ]

    # Obtém dados de modos multiplayer
    data_frame_multiplayer_modes = fetch_multiplayer_modes(games_id_list)

    # Obtém informações gerais do jogo
    data_frame_games = fetch_game_info(games_id_list)

    # Faz o batch fetch das classificações etárias
    unique_age_ratings = data_frame_games["age_ratings"].explode().dropna().unique().tolist()
    age_ratings_df = batch_fetch_age_classifications(unique_age_ratings)

    # Mapeia as classificações etárias para o DataFrame de jogos
    data_frame_games = map_age_classifications(data_frame_games, age_ratings_df)

    # Merge com dados das empresas e modos multiplayer
    df_companies_games = pd.merge(
        df_companies_final,
        data_frame_games_find[["game", "region_name"]].drop_duplicates(subset="game"),
        on="game",
        how="inner",
    )
    df_companies_games_multiplayer = pd.merge(
        df_companies_games.drop_duplicates(subset="game"),
        data_frame_multiplayer_modes[
            ["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]
        ],
        on="game",
        how="left",
    )

    # Combina tudo em um único DataFrame final
    df_join = (
        pd.merge(
            data_frame_games,
            df_companies_games_multiplayer,
            left_on="id",
            right_on="game",
            how="left",
        )
        .drop(columns=["game"])
        .drop_duplicates(subset="id")
    )

    # Gera variáveis dummy
    df_join = generate_dummies(
        df_join, ["platforms_name", "game_modes_name", "player_perspective_name"]
    )

    # Adiciona flag para lançamento global
    df_join["has_global_launch"] = np.where(df_join["region_name"] == "worldwide", 1, 0)
    df_join = df_join.drop(columns=["region_name"])

    return df_join

In [92]:
single = True
year = 2021
game_id = 311813

In [114]:
df_join = game_featurization(single=single,year=year,game_id=game_id)

/Users/josepaulo/data_master_eng_ml/data_master_eng_ml/utils/helpers.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_frame["country"] = data_frame["country"].fillna(1).astype(int)


In [115]:
df_join

,id,name,genres_first,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,onlinecoop,onlinecoopmax,onlinemax,splitscreen,pc,single_player,bird_view_slash_isometric,has_global_launch
0,311813,Waifu Quest 2,point-and-click,False,0,No Rating,1,0,1,Unknown,NaN,NaN,NaN,NaN,1,1,1,1


In [ ]:
df_join = game_featurization(single=single,year=year)

In [91]:
df_join

,id,name,genres_first,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,...,unknown_game_mode,auditory,bird_view_slash_isometric,first_person,side_view,text,third_person,unknown_player_perspectives,virtual_reality,has_global_launch
0,79,Stubbs the Zombie in Rebel Without a Pulse,hack-and-slash-beat-em-up,False,1,18+,4.0,1.0,1.0,North America,...,0,0,0,0,0,0,1,0,0,1
1,386,Final Fantasy XIV Online,role-playing-rpg,False,1,16+,187.0,0.0,962.0,Asia,...,0,0,0,0,0,0,1,0,0,1
2,3241,Cookie Clicker,indie,False,1,All Ages,3.0,0.0,2.0,Unknown,...,0,0,0,0,0,1,0,0,0,1
3,4444,The Immortal,role-playing-rpg,False,1,12+,125.0,0.0,1353.0,North America,...,0,0,0,0,0,0,1,0,0,1
4,5472,50 Cent: Blood on the Sand,shooter,False,1,18+,5.0,0.0,-1.0,Europe,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,311972,Turkey Hero,arcade,False,0,No Rating,1.0,0.0,-1.0,Unknown,...,0,0,0,0,0,0,0,1,0,1
1231,311980,Boxing,sport,False,0,No Rating,3.0,0.0,3.0,Unknown,...,0,0,1,0,0,0,0,0,0,1
1232,225989,Snoot Game,visual-novel,False,0,No Rating,2.0,0.0,2.0,Unknown,...,0,0,0,0,1,1,0,0,0,1
1233,194185,Infinite Links,adventure,False,0,15+,27.0,0.0,1.0,Unknown,...,0,0,1,0,1,0,0,0,0,1


In [ ]:
df_join

,id,name,genres_first,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,...,unknown_game_mode,auditory,bird_view_slash_isometric,first_person,side_view,text,third_person,unknown_player_perspectives,virtual_reality,has_global_launch
0,71,Portal,shooter,False,1,15+,125.0,0.0,1353.0,North America,...,0,0,0,1,0,0,0,0,0,1
1,72,Portal 2,shooter,False,1,12+,86.0,0.0,95.0,North America,...,0,0,0,1,0,0,0,0,0,1
2,4250,Kingdom: The Far Reaches,adventure,False,0,12+,4.0,0.0,39.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
3,13350,Auf Wiedersehen Monty,adventure,False,1,No Rating,40.0,0.0,68.0,Europe,...,0,0,0,0,1,0,0,0,0,1
4,13812,Bear Bovver,arcade,False,0,No Rating,7.0,0.0,9.0,Unknown,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,265905,Rushdown Rivals,hack-and-slash-beat-em-up,False,0,No Rating,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
1283,296814,Genesis II,shooter,False,0,No Rating,6.0,0.0,-1.0,Europe,...,0,0,1,0,0,0,0,0,0,1
1284,294670,Ad Astra,adventure,False,0,No Rating,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,1,0,0,0
1285,229164,Cubic Light,indie,False,0,3+,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_join = main(single=single,year=year)

2024-09-07 17:04:54.838 | INFO     | data_master_eng_ml.dataset:main:107 - Processing dataset...


KeyError: 'company'

In [ ]:
    data_frame_games_find["region_name"] = data_frame_games_find["region"].map(
        region_mapping_inverted
    )
    
    # Processa dados de empresas
    data_frame_companies = process_companies_data(data_frame_companies)


    # Processa dados do jogo para formatação adicional
    data_frame_games = process_game_data(data_frame_games)

In [147]:
from data_master_eng_ml.transformations.api_raw_data import (
    fetch_raw_game_release_dates_batch,
    fetch_raw_game_release_dates_single,
    fetch_raw_involved_companies,
    fetch_raw_companies_info,
    fetch_raw_multiplayer_modes,
    fetch_raw_game_info
)

In [119]:
data_frame_games_find_raw_batch = fetch_raw_game_release_dates_batch(year=year)

2024-09-07 19:33:14.861 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:81 - Fetching game release dates for the year: 2021
2024-09-07 19:33:14.862 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:98 - Constructing the query and initiating data fetch...
2024-09-07 19:33:19.608 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:101 - Data fetched successfully.
2024-09-07 19:33:19.609 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_batch:103 - Returning the resulting DataFrame.


In [153]:
db = client[MONGODB_DATABASE]  # Substitua pelo nome do seu banco de dados

In [154]:
GAME_RELEASE_DATES_RAW_COLLECTION = 'game_release_dates_raw'

In [159]:
save_dataframe_to_mongodb(
    data_frame_games_find_raw_batch,client,MONGODB_DATABASE,GAME_RELEASE_DATES_RAW_COLLECTION
)

Uploading to collection fgame_release_dates_raw: 100%|██████████| 3/3 [00:04<00:00,  1.52s/it]

2024-09-07 20:13:38.323 | INFO     | __main__:save_dataframe_to_mongodb:30 - Data successfully inserted into MongoDB!


In [ ]:
save_dataframe_to_mongodb(
    data_frame_games_find_raw_batch,client,MONGODB_DATABASE,GAME_RELEASE_DATES_RAW_COLLECTION
)

Uploading to collection fgame_release_dates_raw: 100%|██████████| 3/3 [00:04<00:00,  1.52s/it]

2024-09-07 20:13:38.323 | INFO     | __main__:save_dataframe_to_mongodb:30 - Data successfully inserted into MongoDB!


In [121]:
data_frame_games_find_raw_single = fetch_raw_game_release_dates_single(game_id=game_id)

2024-09-07 19:33:57.374 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:41 - Fetching release dates for game ID: 311813
2024-09-07 19:33:57.377 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:58 - Constructing the query and initiating data fetch...
2024-09-07 19:33:57.899 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:61 - Data fetched successfully.
2024-09-07 19:33:57.899 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_release_dates_single:63 - Returning the resulting DataFrame.


In [130]:
games_id_batch = list(set(data_frame_games_find_raw_batch["game"].to_list()))

In [131]:
games_id_list_batch = ",".join(map(str, games_id_batch))

In [125]:
games_id_single = list(set(data_frame_games_find_raw_single["game"].to_list()))

In [126]:
games_id_list_single = ",".join(map(str, games_id_single))

In [136]:
data_frame_companies_find_raw_batch = fetch_raw_involved_companies(games_id_list_batch)
data_frame_companies_find_raw_batch
company_id_list_batch = list(set(data_frame_companies_find_raw_batch["company"].to_list()))


2024-09-07 19:41:21.327 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:121 - Fetching involved companies for game ID:
2024-09-07 19:41:21.328 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:127 - Constructing the query and initiating data fetch...
2024-09-07 19:41:24.589 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:130 - Data fetched successfully.
2024-09-07 19:41:24.589 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:132 - Returning the resulting DataFrame.


In [137]:
data_frame_companies_find_raw_single = fetch_raw_involved_companies(games_id_list_single)
data_frame_companies_find_raw_single
company_id_list_single = list(set(data_frame_companies_find_raw_single["company"].to_list()))


2024-09-07 19:41:25.416 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:121 - Fetching involved companies for game ID:
2024-09-07 19:41:25.416 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:127 - Constructing the query and initiating data fetch...
2024-09-07 19:41:26.316 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:130 - Data fetched successfully.
2024-09-07 19:41:26.317 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_involved_companies:132 - Returning the resulting DataFrame.


In [142]:
data_frame_companies_raw_batch = fetch_raw_companies_info(company_id_list_batch)

2024-09-07 19:45:51.854 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_companies_info:151 - Fetching information for companies with IDs: [47104, 1, 47105, 34817, 47110, 47112, 38924, 55315, 18454, 26, 27, 29, 38942, 55327, 20512, 36897, 8226, 40995, 36, 37, 30758, 38, 30760, 18474, 10282, 2095, 16433, 12340, 16437, 28729, 58, 20538, 30779, 61, 34879, 6209, 66, 47169, 41029, 70, 34886, 72, 18505, 73, 41033, 30796, 16457, 55372, 82, 30802, 55381, 26710, 55382, 22619, 18523, 30821, 104, 28776, 14440, 55405, 110, 36975, 112, 55409, 55408, 43124, 34936, 49274, 43130, 57469, 57470, 129, 51329, 12419, 20611, 34949, 51336, 37002, 37003, 26764, 51342, 10383, 49295, 34960, 51346, 12433, 30868, 12438, 24726, 45211, 4252, 51357, 6303, 41121, 8354, 53409, 6309, 28837, 53414, 41127, 51367, 170, 53419, 8364, 55462, 55463, 24751, 51375, 28847, 22704, 16565, 24760, 22713, 184, 53435, 2238, 22719, 49345, 196, 197, 2245, 18629, 51400, 55497, 10442, 57540, 51404, 49358, 55503, 3912

In [140]:
data_frame_companies_raw_single = fetch_raw_companies_info(company_id_list_single)

2024-09-07 19:45:32.497 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_companies_info:151 - Fetching information for companies with IDs: [56898]
2024-09-07 19:45:32.498 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_companies_info:165 - Constructing the query and initiating data fetch...
2024-09-07 19:45:33.000 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_companies_info:168 - Data fetched successfully.
2024-09-07 19:45:33.000 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_companies_info:171 - Ensuring all expected columns are present in the DataFrame.
2024-09-07 19:45:33.003 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_companies_info:174 - Returning the processed DataFrame.


In [145]:
data_frame_multiplayer_modes_raw_batch = fetch_raw_multiplayer_modes(games_id_list_batch)
data_frame_multiplayer_modes_raw_batch

2024-09-07 19:48:37.643 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:194 - Fetching multiplayer modes for game ID: 155670,270361,270367,270368,163880,172097,155719,262219,106575,155728,79,278610,278612,163926,163927,163928,163929,303194,163930,163932,163934,163935,303201,303202,163939,163937,270437,163938,163954,155763,155766,270462,286854,180372,303260,172198,303274,303275,303276,270506,123054,303278,303288,303292,303293,303300,303301,172230,270537,180430,172242,278759,180458,229610,155900,254209,254211,33027,155917,155918,180495,164118,164124,155949,16687,254257,254258,254260,254261,164160,262476,254290,155994,311650,164198,172397,311666,254326,386,311682,172451,172452,246188,180655,188847,147891,246206,188871,164305,188882,172504,16867,229861,188904,188914,131573,229893,188957,148005,254504,254507,148016,156208,180813,189007,123473,303698,238173,139878,254570,180843,279162,148103,17032,311945,148106,303755,254608,254609,311952,311955,31195

,id,campaigncoop,game,lancoop,offlinecoop,onlinecoop,splitscreen,offlinecoopmax,offlinemax,onlinecoopmax,onlinemax
0,8715,False,101461,False,False,False,False,NaN,NaN,NaN,NaN
1,7522,False,10239,False,False,False,False,NaN,NaN,NaN,NaN
2,10931,False,132637,False,True,False,False,3.0,4.0,0.0,0.0
3,12969,False,142573,False,False,False,False,NaN,NaN,NaN,NaN
4,13406,False,143858,False,True,False,False,3.0,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
114,29010,False,103538,False,False,False,False,NaN,NaN,NaN,50.0
115,29703,False,106435,False,True,False,False,NaN,2.0,NaN,NaN
116,29704,False,106435,False,True,False,False,NaN,2.0,NaN,NaN
117,29435,False,129108,False,True,True,True,4.0,NaN,4.0,NaN


In [146]:
data_frame_multiplayer_modes_raw_single = fetch_raw_multiplayer_modes(games_id_list_single)
data_frame_multiplayer_modes_raw_single

2024-09-07 19:48:39.101 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:194 - Fetching multiplayer modes for game ID: 311813
2024-09-07 19:48:39.102 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:211 - Constructing the query and initiating data fetch...
2024-09-07 19:48:39.477 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:214 - Data fetched successfully.
2024-09-07 19:48:39.477 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:217 - Ensuring all expected columns are present in the DataFrame.
2024-09-07 19:48:39.480 | DEBUG    | data_master_eng_ml.transformations.api_raw_data:fetch_raw_multiplayer_modes:220 - Returning the processed DataFrame.


,campaigncoop,game,lancoop,offlinecoop,offlinecoopmax,offlinemax,onlinecoop,onlinecoopmax,onlinemax,splitscreen


In [148]:
data_frame_games_raw_batch = fetch_raw_game_info(games_id_list_batch)

2024-09-07 19:52:05.971 | INFO     | data_master_eng_ml.transformations.api_raw_data:fetch_raw_game_info:240 - Fetching detailed information for game ID: 155670,270361,270367,270368,163880,172097,155719,262219,106575,155728,79,278610,278612,163926,163927,163928,163929,303194,163930,163932,163934,163935,303201,303202,163939,163937,270437,163938,163954,155763,155766,270462,286854,180372,303260,172198,303274,303275,303276,270506,123054,303278,303288,303292,303293,303300,303301,172230,270537,180430,172242,278759,180458,229610,155900,254209,254211,33027,155917,155918,180495,164118,164124,155949,16687,254257,254258,254260,254261,164160,262476,254290,155994,311650,164198,172397,311666,254326,386,311682,172451,172452,246188,180655,188847,147891,246206,188871,164305,188882,172504,16867,229861,188904,188914,131573,229893,188957,148005,254504,254507,148016,156208,180813,189007,123473,303698,238173,139878,254570,180843,279162,148103,17032,311945,148106,303755,254608,254609,311952,311955,311956,311

In [149]:
data_frame_games_raw_batch

,id,age_ratings,game_modes,genres,involved_companies,name,platforms,player_perspectives,rating,remasters
0,79,"[3435, 56143, 68812, 82150, 110521, 125679, 12...","[1, 2, 3]","[25, 31]","[121739, 134666]",Stubbs the Zombie in Rebel Without a Pulse,"[6, 11, 14, 48, 49, 130]",[2],73.458158,NaN
1,386,"[31733, 110242, 168446, 168447, 168448, 168449...","[1, 2, 3, 5]",[12],"[253758, 275027, 275033, 275055]",Final Fantasy XIV Online,"[6, 9, 14, 48, 167, 169]",[2],81.324771,NaN
2,3241,[112528],[1],"[2, 13, 15, 32]",[8461],Cookie Clicker,"[3, 6, 14, 34, 82]",[5],71.458986,NaN
3,5472,"[2943, 50361, 67909, 167965]","[1, 2, 3]",[5],"[13255, 13257, 95307]",50 Cent: Blood on the Sand,"[9, 12, 49]",[2],58.153653,NaN
4,9254,"[23920, 68850, 89005, 89006, 89008, 89009, 133...",[1],"[31, 32]","[65382, 66501, 66502, 225432]",Subnautica,"[6, 14, 48, 49, 130, 163, 167, 169, 385]","[1, 7]",83.300992,NaN
...,...,...,...,...,...,...,...,...,...,...
1578,37057,"[41773, 142868, 142869, 142870, 142871, 142872]",[1],"[12, 15, 16, 32, 35]","[161475, 161476]",Griftlands,"[3, 6, 14, 48, 49, 130, 169]",[4],76.283069,NaN
1579,125431,"[46606, 52207, 73180, 92873, 125110, 125111]",[1],"[8, 9, 13, 31]","[193517, 193518]",I Am Fish,"[6, 49, 169]",[2],67.787634,NaN
1580,123473,"[44725, 56082, 98095, 114408, 127925, 150818]",[1],"[12, 31, 32, 34]",[116871],Arietta of Spirits,"[6, 48, 49, 130, 167]",[3],NaN,NaN
1581,27413,"[94234, 106869, 108779, 117788, 125286, 164238]",[1],"[2, 31, 32]","[249847, 249848]",Genesis Noir,"[6, 14, 49, 130]","[1, 2, 4]",68.828520,NaN


In [57]:
data_frame_games = fetch_game_info(games_id_list)

In [59]:
# Faz o batch fetch das classificações etárias
unique_age_ratings = data_frame_games['age_ratings'].explode().dropna().unique().tolist()

In [60]:
age_ratings_df = batch_fetch_age_classifications(unique_age_ratings)

In [62]:
# Mapeia as classificações etárias para o DataFrame de jogos
data_frame_games = map_age_classifications(data_frame_games, age_ratings_df)

In [64]:
companies_join = pd.merge(data_frame_companies_find, data_frame_companies, left_on="company", right_on="id", how="inner")

df_companies_final = companies_join[["game", "games_developed", "has_parents", "games_published", "continent_name"]]

In [65]:
# Merge com dados das empresas e modos multiplayer
df_companies_games = pd.merge(df_companies_final, data_frame_games_find[["game", "region_name"]].drop_duplicates(subset="game"), on="game", how="inner")
df_companies_games_multiplayer = pd.merge(df_companies_games.drop_duplicates(subset="game"), data_frame_multiplayer_modes[["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]], on="game", how="left")

# Combina tudo em um único DataFrame final
df_join = pd.merge(data_frame_games, df_companies_games_multiplayer, left_on="id", right_on="game", how="left").drop(columns=["game"]).drop_duplicates(subset="id")


In [66]:
df_join

,id,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,region_name,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,71,Portal,[first-person],"[mobile, pc, modern_console]",shooter,[single-player],False,1,15+,125.0,0.0,1353.0,North America,worldwide,NaN,NaN,NaN,NaN
1,72,Portal 2,[first-person],"[pc, modern_console]",shooter,"[multiplayer, co-operative, single-player, spl...",False,1,12+,86.0,0.0,95.0,North America,worldwide,True,2.0,0.0,True
6,4250,Kingdom: The Far Reaches,[third-person],"[less_common_portable_console, unknown_platfor...",adventure,[single-player],False,0,12+,4.0,0.0,39.0,Unknown,worldwide,NaN,NaN,NaN,NaN
7,6614,Zero Escape: Virtue's Last Reward,"[text, first-person]","[portable_console, pc, modern_console]",visual-novel,[single-player],False,1,18+,1.0,0.0,132.0,Europe,worldwide,NaN,NaN,NaN,NaN
8,7046,Factorio,[bird-view-slash-isometric],"[pc, modern_console]",strategy,"[multiplayer, co-operative, single-player]",False,1,12+,2.0,0.0,2.0,Europe,worldwide,True,65535.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,296814,Genesis II,[bird-view-slash-isometric],"[portable_console, unknown_platforms_name]",shooter,[single-player],False,0,No Rating,6.0,0.0,-1.0,Europe,worldwide,NaN,NaN,NaN,NaN
1677,230728,Four Corners for Playdate,[unknown_player_perspectives],[unknown_platforms_name],arcade,[unknown_game_mode],False,0,No Rating,9.0,0.0,9.0,Unknown,worldwide,NaN,NaN,NaN,NaN
1678,294670,Ad Astra,"[third-person, first-person]",[pc],adventure,"[multiplayer, co-operative, single-player]",False,0,No Rating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,229164,Cubic Light,[unknown_player_perspectives],[modern_console],indie,[single-player],False,0,3+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Gera variáveis dummy
df_join = generate_dummies(df_join, ["platforms_name", "game_modes_name", "player_perspective_name"])

# Adiciona flag para lançamento global
df_join["has_global_launch"] = np.where(df_join["region_name"] == "worldwide", 1, 0)
df_join = df_join.drop(columns=["region_name"])

In [69]:
df_join

,id,name,genres_first,has_remaster,target,age_rating_group,games_developed,has_parents,games_published,continent_name,...,unknown_game_mode,auditory,bird_view_slash_isometric,first_person,side_view,text,third_person,unknown_player_perspectives,virtual_reality,has_global_launch
0,71,Portal,shooter,False,1,15+,125.0,0.0,1353.0,North America,...,0,0,0,1,0,0,0,0,0,1
1,72,Portal 2,shooter,False,1,12+,86.0,0.0,95.0,North America,...,0,0,0,1,0,0,0,0,0,1
2,4250,Kingdom: The Far Reaches,adventure,False,0,12+,4.0,0.0,39.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
3,6614,Zero Escape: Virtue's Last Reward,visual-novel,False,1,18+,1.0,0.0,132.0,Europe,...,0,0,0,1,0,1,0,0,0,1
4,7046,Factorio,strategy,False,1,12+,2.0,0.0,2.0,Europe,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,296814,Genesis II,shooter,False,0,No Rating,6.0,0.0,-1.0,Europe,...,0,0,1,0,0,0,0,0,0,1
1268,230728,Four Corners for Playdate,arcade,False,0,No Rating,9.0,0.0,9.0,Unknown,...,1,0,0,0,0,0,0,1,0,1
1269,294670,Ad Astra,adventure,False,0,No Rating,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,1,0,0,0
1270,229164,Cubic Light,indie,False,0,3+,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0


In [40]:
df_join.to_csv(f'twitch_api_data_{ano}.csv',header=True,index=False)